1. 세션의 지속 시간을 분으로 계산하고 가장 긴 지속시간을 출력하시오(반올림 후 총 분만 출력)
2. 가장 많이 머무른 Page를 찾고 그 페이지의 머문 평균 시간을 구하시오 (반올림 후 총 시간만 출력)
3. 사용자들이 가장 활발히 활동하는 시간대(예: 새벽, 오전, 오후, 저녁)를 분석하세요. 이를 위해 하루를 4개의 시간대로 나누고 각 시간대별로 가장 많이 시작된 세션의 수를 계산하고, 그 중에 가장 많은 세션 수를 출력하시오
      - 새벽: 0시 부터 6시 전
      - 오전: 6시 부터 12시 전
      - 오후: 12 부터 18시 전
      - 저녁: 18시 부터 0시 전
4. user가 가장 많이 접속 했던 날짜를 출력하시오. (예, 2023-02-17)

In [365]:
import pandas as pd

data_path = 'C:\Engineer_Big_Data_Analysis\DATAS\kaggle_BDCKR\website.csv'
df = pd.read_csv(data_path)
print(df.shape)
df

(9601, 4)


,UserID,StartTime,EndTime,Page
0,1,2023-07-02 08:32:00,2023-07-02 12:41:00,Page1
1,1,2023-05-16 11:37:00,2023-05-16 13:29:00,Page5
2,1,2023-05-05 03:39:00,2023-05-05 05:48:00,Page2
3,1,2023-05-04 23:04:00,2023-05-05 01:54:00,Page3
4,1,2023-09-04 11:27:00,2023-09-04 14:09:00,Page1
...,...,...,...,...
9596,1202,2023-06-26 09:43:00,2023-06-26 13:43:00,Page3
9597,1202,2023-11-17 02:21:00,2023-11-17 04:31:00,Page3
9598,1202,2023-03-07 14:56:00,2023-03-07 17:27:00,Page2
9599,1202,2023-07-24 03:22:00,2023-07-24 06:58:00,Page2


In [366]:
# 우선 데이터 타입 확인
print(df.dtypes)
print('-'*50)
# StartTime, EndTime 형식을 datetime으로 바꾸기
df.StartTime = pd.to_datetime(df.StartTime)
df.EndTime = pd.to_datetime(df.EndTime)
# 다시 확인
print(df.dtypes)

UserID        int64
StartTime    object
EndTime      object
Page         object
dtype: object
--------------------------------------------------
UserID                int64
StartTime    datetime64[ns]
EndTime      datetime64[ns]
Page                 object
dtype: object


In [367]:
# 1. 세션의 지속 시간을 분으로 계산하고 가장 긴 지속시간을 출력하시오(반올림 후 총 분만 출력)
# EndTime - StartTime 으로 DwellTime 계산해서 열에 추가
df['DwellTime'] = round((df['EndTime'] - df['StartTime']).dt.total_seconds()/60)
display(df.head())
print(df.DwellTime.max())

,UserID,StartTime,EndTime,Page,DwellTime
0,1,2023-07-02 08:32:00,2023-07-02 12:41:00,Page1,249.0
1,1,2023-05-16 11:37:00,2023-05-16 13:29:00,Page5,112.0
2,1,2023-05-05 03:39:00,2023-05-05 05:48:00,Page2,129.0
3,1,2023-05-04 23:04:00,2023-05-05 01:54:00,Page3,170.0
4,1,2023-09-04 11:27:00,2023-09-04 14:09:00,Page1,162.0


300.0


In [368]:
# 2. 가장 많이 머무른 Page를 찾고 그 페이지의 머문 평균 시간을 구하시오 (반올림 후 총 시간만 출력)
agg_df = df.groupby('Page').DwellTime.agg(['sum','mean']).sort_values('mean', ascending=False)
print(f'가장 많이 머무른 페이지는 {agg_df.iloc[0].name}\n총 DwellTime은 {round(agg_df.iloc[0, 0])}분\n평균 DwellTime은 {round(agg_df.iloc[0,1]/60)}시간')

가장 많이 머무른 페이지는 Page5
총 DwellTime은 301903분
평균 DwellTime은 3시간


In [369]:
# 3. 사용자들이 가장 활발히 활동하는 시간대(예: 새벽, 오전, 오후, 저녁)를 분석하세요.
# 이를 위해 하루를 4개의 시간대로 나누고 각 시간대별로 가장 많이 시작된 세션의 수를 계산하고,
# 그 중에 가장 많은 세션 수를 출력하시오
#       - 새벽: 0시 부터 6시 전
#       - 오전: 6시 부터 12시 전
#       - 오후: 12 부터 18시 전
#       - 저녁: 18시 부터 0시 전

# 복제 df 만들기
df_start = df.copy()
# UserID 필요없음
del df_start['UserID']
df_start

,StartTime,EndTime,Page,DwellTime
0,2023-07-02 08:32:00,2023-07-02 12:41:00,Page1,249.0
1,2023-05-16 11:37:00,2023-05-16 13:29:00,Page5,112.0
2,2023-05-05 03:39:00,2023-05-05 05:48:00,Page2,129.0
3,2023-05-04 23:04:00,2023-05-05 01:54:00,Page3,170.0
4,2023-09-04 11:27:00,2023-09-04 14:09:00,Page1,162.0
...,...,...,...,...
9596,2023-06-26 09:43:00,2023-06-26 13:43:00,Page3,240.0
9597,2023-11-17 02:21:00,2023-11-17 04:31:00,Page3,130.0
9598,2023-03-07 14:56:00,2023-03-07 17:27:00,Page2,151.0
9599,2023-07-24 03:22:00,2023-07-24 06:58:00,Page2,216.0


In [370]:
# apply에 쓸 시간대 반환 함수 만들고
def time_trim(h):
    if 0 <= h < 6 : return '새벽'
    elif 6 <= h < 12 : return '오전'
    elif 12 <= h < 18 : return '오후'
    elif 18 <= h < 24 : return '저녁'
# 적용해서 열 추가하기
df_start['TimeSlot'] = df_start.StartTime.dt.hour.apply(time_trim)
display(df_start)
print('-'*50)
# 시간대별 개수 집계
display(df_start.TimeSlot.value_counts())
# 개수만 확인
df_start.TimeSlot.value_counts().max()

,StartTime,EndTime,Page,DwellTime,TimeSlot
0,2023-07-02 08:32:00,2023-07-02 12:41:00,Page1,249.0,오전
1,2023-05-16 11:37:00,2023-05-16 13:29:00,Page5,112.0,오전
2,2023-05-05 03:39:00,2023-05-05 05:48:00,Page2,129.0,새벽
3,2023-05-04 23:04:00,2023-05-05 01:54:00,Page3,170.0,저녁
4,2023-09-04 11:27:00,2023-09-04 14:09:00,Page1,162.0,오전
...,...,...,...,...,...
9596,2023-06-26 09:43:00,2023-06-26 13:43:00,Page3,240.0,오전
9597,2023-11-17 02:21:00,2023-11-17 04:31:00,Page3,130.0,새벽
9598,2023-03-07 14:56:00,2023-03-07 17:27:00,Page2,151.0,오후
9599,2023-07-24 03:22:00,2023-07-24 06:58:00,Page2,216.0,새벽


--------------------------------------------------


새벽    2447
저녁    2444
오전    2406
오후    2304
Name: TimeSlot, dtype: int64

2447

In [371]:
# 시간대별 가장 많이 들어가는 페이지 확인
pd.DataFrame(df_start.groupby(['TimeSlot', 'Page']).StartTime.count()).sort_values(by=['TimeSlot', 'StartTime'], ascending=False)

StartTime
TimeSlot Page            
저녁       Page5        512
         Page1        498
         Page3        495
         Page4        491
         Page2        448
오후       Page3        472
         Page1        466
         Page2        459
         Page5        455
         Page4        452
오전       Page5        538
         Page3        500
         Page4        463
         Page1        453
         Page2        452
새벽       Page4        526
         Page3        494
         Page1        480
         Page5        479
         Page2        468

In [372]:
# 그룹핑 한 것에서 세션수가 가장 많은 Page와 세션수
target_row = pd.DataFrame(df_start.groupby(['TimeSlot', 'Page']).StartTime.count()).sort_values('StartTime', ascending=False).iloc[0]
print(target_row)
print(target_row.name[1], target_row.StartTime)

StartTime    538
Name: (오전, Page5), dtype: int64
Page5 538


In [364]:
# 4. user가 가장 많이 접속 했던 날짜를 출력하시오. (예, 2023-02-17)
df_user = df.copy()
df_user.set_index('StartTime', inplace=True)
df_user

,UserID,EndTime,Page,DwellTime
StartTime,,,,
2023-07-02 08:32:00,1,2023-07-02 12:41:00,Page1,249.0
2023-05-16 11:37:00,1,2023-05-16 13:29:00,Page5,112.0
2023-05-05 03:39:00,1,2023-05-05 05:48:00,Page2,129.0
2023-05-04 23:04:00,1,2023-05-05 01:54:00,Page3,170.0
2023-09-04 11:27:00,1,2023-09-04 14:09:00,Page1,162.0
...,...,...,...,...
2023-06-26 09:43:00,1202,2023-06-26 13:43:00,Page3,240.0
2023-11-17 02:21:00,1202,2023-11-17 04:31:00,Page3,130.0
2023-03-07 14:56:00,1202,2023-03-07 17:27:00,Page2,151.0


In [394]:
# 데일리 단위로 리샘플링
str(df_user.resample('D').count().sort_values('UserID', ascending=False).index[0].date())

'2023-03-28'